# Combinando un par de modelos

In [ ]:
!pip install sacremoses

In [ ]:
# Librerías Generales a utilizar
import torch
import os
from IPython.display import Image

# Librería de modelo opus de Helsinki
from transformers import AutoTokenizer as AT, MarianMTModel as MT

# Librería para Stable Diffusion
from diffusers import StableDiffusionPipeline as SDP

In [ ]:
# Descargar y configuración del modelo de traducción
nombreDeModelo = f"Helsinki-NLP/opus-mt-es-en"
traductor = MT.from_pretrained(nombreDeModelo)
tokenizador = AT.from_pretrained(nombreDeModelo)

In [ ]:
# Descarga de modelo preentrenado de Stable Difussion
model_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"
sd_pipeline = SDP.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)
sd_pipeline = sd_pipeline.to( "cuda" )

In [ ]:
# función que facilitará la solicitud de traducción
def opus_traduce( texto_a_traducir ):
    tokens = tokenizador.tokenize( texto_a_traducir )
    lote = tokenizador( texto_a_traducir, return_tensors="pt")
    identificadores = traductor.generate(**lote)
    traduccion = tokenizador.batch_decode(
        identificadores,
        skip_special_tokens=True
    )
    return traduccion[0]

In [ ]:
# Función que faicilitará la generación de imágenes
def generar_imagen( descripcion ) :
    lo_que_no_queremos = """
    Bad anatomy,Bad hands,Amputee,Missing fingers,Missing hands,Missing limbs,
    Missing arms,Extra fingers,Extra hands,Extra limbs,Mutated hands,Mutated,
    Mutation,Multiple heads,Malformed limbs,Disfigured,Poorly drawn hands,
    Poorly drawn face,Long neck,Fused fingers,Fused hands,Dismembered,
    Duplicate,Improper scale,Ugly body,Cloned face,Cloned body,Error,
    Gross proportions,Body horror,Too many fingers,simple_background,Pixelated,
    Croped,Out of frame,Out of focus,Dehydrated,lowest quality,Collage,Pixel,
    Bad proportions,extra digits,lowers,username,Disfigured,Disgusting,
    Low res,Low quality,Jpeg,Jpeg artifacts,artist name,error,watermark,
    signature,text,mutated hands,extra digit,fewer digits,worst quality,blury
    """
    generado = sd_pipeline(
        descripcion,
        negative_prompt = lo_que_no_queremos
    )
    return generado.images[0]

In [ ]:
# probar la traducción
prueba_texto = """
imagen con estilo de pintura Van Gogh de un hombre mago,
de temática medieval, usando una toga gris y barba larga,
sosteniendo un báculo brillante en su mano derecha,
caminando por un bosque en la noche
"""
texto_traducido = opus_traduce( prueba_texto )
print( f"Español:\n{prueba_texto}"    )
print( f"Inlgés :\n{texto_traducido}" )

In [ ]:
# usar la traducción para generar la imagen
prueba_imagen = texto_traducido
imagen_generada = generar_imagen( prueba_imagen )
imagen_generada.save( "./mi_creacion.jpg" )
Image("./mi_creacion.jpg")